# Der erwartungswert im allgemeinen Fall für das Wort $W$
Dieser Notebook zielt auf die Berechung des Erwartungswerts einer iterierten Summe von AR(1) mit einem beliebigen Wort $W=i_1i_2\dotsi_k \in \mathbb N^*$ aufbauend hierauf des Isserlies' Satzes und mit Hilfe der Proposition 7.3.20 ab, die den Erwartungswert einer iterierten Summe von AR(1) mit dem Wort $W=1234$ berechnet hat, Diese Notebook erreicht das Ziel durch:
- Berrechnen $\mathbf P^2_{\mathbb M\{i_1,i_2,\dots,i_k\}}$  die Menge der Matchings der Multimenge $\mathbb M\{i_1,i_2,\dots,i_k\}$, diese Menge taucht in die folgende Formel auf 
\begin{align*}
    &\mathbf{E}[\langle \mathsf{ISS}(X),i_1\dots i_k\rangle]=\sum_{1\le j_k<\dots<j_2<j_1\le n}\sum_{p\in P^2_{\mathbb M[i_1i_2\dots i_k]}}\prod_{(l,l^\prime)\in p}\mathbf{Cov}(\Delta X_{j_l},\Delta X_{j_{l^\prime}})
\end{align*}, Dies wird durch die Beziehung zwischen der Menge der Matchings der Multimenge $\mathbb M\{i_1,i_2,\dots,i_k\}$ und den Mengen der Matchings bestimmter vollständiger k-partiten Graphen getan, die im Lemma 6.2.5. in der Masterarbeit bewiesen wurde. Für ein Wort $W=i_1i_2\dots i_k $ definiere ich das Kartesische Produkt $\mathsf{PROD}=[\lfloor\frac{i_1}{2}\rfloor]\times[\lfloor\frac{i_2}{2}\rfloor]\times\dots\times[\lfloor\frac{i_k}{2}\rfloor]$. Dann gilt:
  \begin{equation}P^2_{\mathbb M[i_1i_2\dots i_k]}=\bigcup_{ \mathsf{prod}\in \mathsf{PROD} }(11)^{\mathsf{prod}_1}\star (22)^{\mathsf{prod}_2}\star\dots\star (kk)^{\mathsf{prod}_k}\star P^2_{\mathbb K_{W-2\mathsf{prod}}}\end{equation}
  Um den Wert der Elementen der Menge $P^2_{\mathbb M[i_1i_2\dots i_k]}$ in der iterierten Summe zu berechnen, werden paar Funktionen gebraucht.

In [1]:
import numpy as np
import pandas as pd 
import copy
import itertools as it
from itertools import product as carprod
import scipy
from scipy.special import comb
import pprint
from scipy.special import factorial
from numpy import product
import fractions

In [2]:
def without_max(A):#a tool used to calculate the frequency fq
    A_ref=copy.copy(A)
    for i in range(len(A)):
       if i ==np.argmax(A):
            A_ref[i]=0
    return(A_ref)

In [3]:
def rounded_halves(A): # a tool used to find the set over which we summe by the Cartesian product
    X=[]
    for i in range(len(A)):
        X.append(list(range(int(np.floor(A[i]/2)+1))))
    return(X)

### 1-Die erste Funktion findet die Menge der perfekten Matchings vom vollständigen k-partiten Graphen $\mathbf{P}^2_{\mathbb K_{W}}$, indem:
- Die folgende Funktion alle Möglichkeiten findet, die Knoten der maximalen Klasse zu verbinden (zu paaren) 
- Dann wiederholt die Funktion den gleichen Schritt für die maximale Klasse aus den verbleibenden nicht-verbundenen Klassen.
- Die Funktion wiederholt den Schritt, bis die Anzahl der Knoten in der maximalen Klasse der verbleibenden Klassen gleich die Anzahl der Knoten aushalb der maximalen Klasse der verbleibenden Klassen

In [4]:
def all_sum_rec(target, current_sum, start, output, result):
 if current_sum == target:
   output.append(copy.copy(result))

 for i in range(start, target):
   temp_sum = current_sum + i
   if temp_sum <= target:
     result.append(i)
     all_sum_rec(target, temp_sum, i, output, result)
     result.pop()
   else:
     return
def all_sum(target,leng): #the function finds all arrays of a certain length whose sum is equal to a certain number.
 output = []
 result = []
 all_sum_rec(target, 0, 1, output, result)
 output1 =np.zeros((len(output),leng),dtype=int)
 for i in range(len(output)):
        a=np.array(output[i])
        if len(a) <= leng:
           for j in range(len(a)):
              output1[i,j]=a[j]
 Temp_Matching=np.zeros((0,),dtype=int)
 for i in range(len(output1)):
    if sum(output1[i])!=0 and (np.sum(output1[i])-2*np.max(output1[i]))>=0:
          Temp_Matching = np.concatenate([Temp_Matching,output1[i]],axis=0)
 Temp_Matching=np.reshape(Temp_Matching,(int(len(Temp_Matching)/leng),leng))
 return Temp_Matching
#this function finds all possible ways to connect the nodes of the maximum class as one array and the remaining ones as another array
def Matching_with_and_without_max(W): 
    D=np.sum(W)-(2*np.max(W))
    if D>0:
        A=all_sum(D,len(W)-1)
        Matching_without_max=np.zeros((0,len(W)),dtype=int)
        for x in A:
            M=list(it.permutations(x))
            for m in M:
                v=np.array(m)
                v=np.insert(v,np.argmax(W),0,axis=0)
                if (v<=W).all():
                    Matching_without_max=np.concatenate((Matching_without_max,v.reshape(1,len(W))),axis=0)
    elif D==0:
        Matching_without_max=np.zeros((1,len(W)),dtype=int)
    Matching_without_max=np.unique(Matching_without_max,axis=0)
    Matching_with_max=np.zeros([0,len(W)],dtype=int)
    for x in Matching_without_max:
        b=W-x
        Matching_with_max=np.concatenate([Matching_with_max,b.reshape(1,len(W))],axis=0)
    M1=np.zeros((0,len(W)),dtype=int)
    M=[]
    for i in range(len(Matching_with_max)):
        M1=np.concatenate((Matching_with_max[i].reshape(1,len(W)),Matching_without_max[i].reshape(1,len(W))),axis=0)
        M.append(M1)
    return(M)

def condition(K):# The condition for ending that while loop
    index=True
    for x in K:
        if np.sum(x[-1])-(2*np.max(x[-1]))!=0:
            index=False
            break
    return(index)

def extension_of_matching(L):
    K=[]
    for x in L:
        if np.sum(x[-1])-(2*np.max(x[-1]))!=0:
            z=Matching_with_and_without_max(x[-1])
            for y in z:
                G=np.zeros((0,len(y[0])),dtype=int)
                G=np.concatenate((x[:-1],y[0].reshape(1,len(y[0])),y[1].reshape(1,len(y[0]))),axis=0)
                K.append(G)
        else:
            K.append(x)
    return(K)

def find_all_matching(W): # the function to find all possible matchings of a complete k-partite graph as a Matrix. 
    secondstep=Matching_with_and_without_max(W)
    index=condition(secondstep)
    while index==False:
          secondstep=extension_of_matching(secondstep)
          index=condition(secondstep)
    return secondstep

### 2-Die zweite Funktion berechnet die Frequenz für jedes Matching aus der Menge $\mathbf{P}^2_{\mathbb K_{W}}$: 
\begin{equation}\mathfrak{fq}_{p,\mathbb K_{W}}:=\frac{W!}{p_{12}!p_{13}!\dots p_{1k}!p_{23}!\dots p_{(k-1)k}!}\end{equation} 
Die Frequenz eines Matching in einem vollständigen k-partiten Graphen (mit labelten Klassen und unlabelten Knoten) $\mathbb K_{W}$ ist die Anzahl, wie oft das Matching in der Menge $\mathbf{P}^2_{\mathbb K_{W}}$ ersheint

In [5]:
def frequency_of_Matching(All_Matching,L):
    frequency_of_Matching=np.zeros((0,1))
    for x in All_Matching:
        fr=np.ones((1,1))
        for y in x:
            F=without_max(y)
            fr*=product(factorial(F))
        frequency_of_Matching=np.concatenate((frequency_of_Matching,L/fr))
    return(frequency_of_Matching)

 ### 3-Die dritte Funktion berechnet für jedes Matching aus der Menge $\mathbf{P}^2_{\mathbb K_{W}}$ das Bild, das aus der Menge der Projektionen der Dyck-Wörter auf $W$ herkommt,  durch die Funktion $\mathbf h$, die wie folgend definiert ist:
\begin{align*}&p=(12)^{p_{12}}(13)^{p_{13}}\dots (1k)^{p_{1k}}(23)^p_{23}\dots (2k)^{p_{2k}}\dots (k-1k)^{p_{k-1k}}\\&\mathbf h(p)=[+{\sum_{\ell>1}^kp_{1\ell}}][-{p_{12}}+{\sum_{\ell>2}^kp_{2\ell}}]\dots[-{\sum_{\ell=1}^{l-1}p_{\ell l}}+{\sum_{\ell>l}^kp_{l\ell}}]\dots[-{\sum_{\ell=1}^{k-1}p_{\ell k}}]\in \mathfrak{DW}\end{align*}
Wobei $\mathfrak{DW}$ die Menge der Projeketionen der Dyck-Wörter auf dem Wort $W$.

In [6]:
def partialDyckWords(M):
    DyckWords=np.zeros(M.shape,dtype=int)
    k=np.argmax(M)
    for j in range(M.shape[0]):
         DyckWords[j]=int(M[j]*np.sign(k-j))
    DyckWords[k]=int(-np.sum(DyckWords))
    return(DyckWords)

def DYCKWORDS(Matching,W):
    DW=np.zeros((0,len(W)),dtype=int)
    for i in range(len(Matching)):
        DW_part=0
        for j in range(len(Matching[i])):
            DW_part=DW_part+partialDyckWords(Matching[i][j])
        DW=np.concatenate((DW,DW_part.reshape(1,len(Matching[0][0]))),axis=0)
    return(DW)

### 4-Die vierte Funktion (FQ_dw_W) findet alle Matchings aus $\mathbf{P}^2_{\mathbb K_{W}}$ der Menge der perfekten Matchings des vollständigen k-partiten Graphen, die eine identische Projektion haben, und summiert ihre Frequenzen 

In [7]:
def FQ_dw_W(DW,frequency,Unique):
    Frequency_of_dw_in_W=np.zeros((0,1))
    for x in  Unique:
        frequency_of_dw_in_W=np.zeros((1,1))
        for i in range(len(DW)):
            if (x==DW[i]).all():
                frequency_of_dw_in_W=frequency_of_dw_in_W+frequency[i]
        Frequency_of_dw_in_W=np.concatenate((Frequency_of_dw_in_W,frequency_of_dw_in_W),axis=0)    
    return(Frequency_of_dw_in_W)

### 5-Die fünfte Funktion gibt alle Projektionen (die Bilder aller Matchings aus der Menge $\mathbf{P}^2_{\mathbb K_{W}}$ durch $\mathbf h$)  und die Frequenz $\mathfrak{fq}_{(\mathfrak{dw},W)}$ einer Dyckwort-Projektion in einem Graphen $\mathbb k_{W}$.

In [8]:
def projektions_of_Dyckwords_with_frequeny_on(W):
    L=product(factorial(W))
    All_Matching=find_all_matching(W)
    frequency=frequency_of_Matching(All_Matching,L)
    DW=DYCKWORDS(All_Matching,W)
    Unique_DW=np.unique(DW,axis=0)
    Frequency_of_dw_in_W=FQ_dw_W(DW,frequency,Unique_DW)
    return(Unique_DW,Frequency_of_dw_in_W)

#### Für ein Wort $W=i_1i_2\dots i_k $ definiere ich das Kartesische Produkt $\mathsf{PROD}=[\lfloor\frac{i_1}{2}\rfloor]\times[\lfloor\frac{i_2}{2}\rfloor]\times\dots\times[\lfloor\frac{i_k}{2}\rfloor]$. Dann gilt:
  \begin{equation}P^2_{\mathbb M[i_1i_2\dots i_k]}=\bigcup_{ \mathsf{prod}\in \mathsf{PROD} }(11)^{\mathsf{prod}_1}\star (22)^{\mathsf{prod}_2}\star\dots\star (kk)^{\mathsf{prod}_k}\star P^2_{\mathbb K_{W-2\mathsf{prod}}}\end{equation}

### Die folgende Funktion wiederholt die fünf Funktionen und findet für alle $\forall \mathsf{prod}\in \mathsf{PROD}$ eine List von den Projektionen und deren Frequenzen 

In [9]:
def All_Dyck_Words(W):
    X=rounded_halves(W)
    Graphs = [i for i in it.product(*X)]
    ALL_DYCK_WORDS=[]
    for x in Graphs:
        x1=np.array(x)
        w=W-2*x1
        p=np.array([int(np.sum(w)/2)])
        D=np.sum(w)-(2*np.max(w))
        if D>=0:
            [Unique_DW,Frequency_of_dw_in_W]=projektions_of_Dyckwords_with_frequeny_on(w)
            x1_factorial=product(factorial(x1))
            w_factorial=product(factorial(w))
            coefficient=Frequency_of_dw_in_W/(x1_factorial*w_factorial)
            ALL_DYCK_WORDS.append([p,coefficient,Unique_DW])
    return(ALL_DYCK_WORDS)

### - Die folgende Funktion durchsucht die Ergebnisliste der letzten Funktion nach identischen Projektionen und zieht diese als gemaeinsamen Faktor und baut aus den Frequenzen und der Defferenz zwischen dem $\mathsf{prod}$ und das Wort $W$ ein Polynom auf:
\begin{align*}\mathcal P(\varphi,\mathfrak{dw}):=\sum_{w_{\mathfrak{dw}} \in \mathfrak W_{\mathfrak{dw}}}\frac{\mathfrak{fq}_{(\mathfrak{dw},w_{\mathfrak{dw}})}}{\frac{W-w_{\mathfrak{dw}}}{2}! w_{\mathfrak{dw}}!}(\frac{- (1-\varphi)}{ \varphi})^{\frac{w_{\mathfrak{dw}}}{2}}\end{align*}
 \begin{equation*}\operatorname{E}[\langle\operatorname{ISS}(X),W\rangle]=W!(\frac{\sigma^2}{1+\varphi})^{\frac{W}{2}}\sum_{\mathfrak{dw} \in \mathfrak{DW}}P(\varphi,\mathfrak{dw})\langle \operatorname{ISS}(Z),\mathfrak{dw} \rangle\end{equation*}

In [10]:
def the_expected_value_of_word(W):
    All_Dyck_Words_withcoefficient=All_Dyck_Words(W)
    the_set_of_all_projektion=np.zeros((0,len(W)))
    for i in range(len(All_Dyck_Words_withcoefficient)):
        the_set_of_all_projektion=np.concatenate((the_set_of_all_projektion,All_Dyck_Words_withcoefficient[i][2]),axis=0)
    unique_the_set_of_all_projektion=np.unique(the_set_of_all_projektion,axis=0)
    the_expected_value_of_word_W=[]
    for x in unique_the_set_of_all_projektion:
        power_and_coefficent_polynomial=[]
        for y in All_Dyck_Words_withcoefficient:
            for j in range(len(y[2])):
                if (x==y[2][j]).all():
                    power_and_coefficent_polynomial.append((fractions.Fraction(y[1][j][0]).limit_denominator(),y[0][0]))
        the_expected_value_of_word_W.append([power_and_coefficent_polynomial,x.astype(int)])
    return(the_expected_value_of_word_W)

In [11]:
W=np.array([1,2,3,4])
the_expected_value_of_word_W=the_expected_value_of_word(W)

In [12]:
Df_the_expected_value_of_word_W=pd.DataFrame(the_expected_value_of_word_W,columns=['(fracture,Power_of_the_constant)','Dyckword'])

### Aus der folgenden Data Frame kann die folgende Formel bilden:
\begin{align*}\mathcal P(\varphi,\mathfrak{dw}):=\sum_{w_{\mathfrak{dw}} \in \mathfrak W_{\mathfrak{dw}}}\frac{\mathfrak{fq}_{(\mathfrak{dw},w_{\mathfrak{dw}})}}{\frac{W-w_{\mathfrak{dw}}}{2}! w_{\mathfrak{dw}}!}(\frac{- (1-\varphi)}{ \varphi})^{\frac{w_{\mathfrak{dw}}}{2}}\end{align*}
 \begin{equation*}\operatorname{E}[\langle\operatorname{ISS}(X),W\rangle]=W!(\frac{\sigma^2}{1+\varphi})^{\frac{W}{2}}\sum_{\mathfrak{dw} \in \mathfrak{DW}}P(\varphi,\mathfrak{dw})\langle \operatorname{ISS}(Z),\mathfrak{dw} \rangle\end{equation*}
 ### Aus der ersten Spalte bildet man das Polynum $ P(\varphi,\mathfrak{dw})$,wobei jeder Term vom Polynom wird aus einem Paar der Spalte gebildet.
\begin{align*} \text{wobei fracture}=\frac{\mathfrak{fq}_{(\mathfrak{dw},w_{\mathfrak{dw}})}}{\frac{W-w_{\mathfrak{dw}}}{2}! w_{\mathfrak{dw}}!},\text{ Power_of_the_constant}= \frac{w_{\mathfrak{dw}}}{2} \end{align*}. 
### Die zweite Spalte ist der Beitrag in der iterierten Summe auf der rechten Seite Dyckword=$\mathfrak{dw}$.

In [13]:
Df_the_expected_value_of_word_W

,"(fracture,Power_of_the_constant)",Dyckword
0,"[(1/2, 2), (1/2, 1)]","[1, 0, -1, 0]"
1,"[(1/2, 4), (1, 3), (1/2, 3), (1, 2)]","[1, 0, 1, -2]"
2,"[(1/6, 5), (1/6, 4)]","[1, 0, 3, -4]"
3,"[(1/4, 3)]","[1, 2, -3, 0]"
4,"[(3/2, 4), (3/2, 3)]","[1, 2, -1, -2]"
5,"[(3/4, 5), (1/2, 4)]","[1, 2, 1, -4]"
